In [ ]:
import smtplib
import imaplib
import os
import email
import random
import string
from email.message import EmailMessage
from email.header import decode_header
from email.utils import parseaddr
from dotenv import load_dotenv

load_dotenv()

EMAIL_ADDRESS = os.getenv("EMAIL_ADDRESS")
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")
SMTP_SERVER = os.getenv("SMTP_SERVER")
SMTP_PORT = int(os.getenv("SMTP_PORT"))
IMAP_SERVER = os.getenv("IMAP_SERVER")

def generate_random_text(length=20):
    return ''.join(random.choice(string.ascii_letters) for _ in range(length))

def create_pdf(filename):
    content = generate_random_text(100)
    with open(filename, "wb") as f:
        f.write(bytes(f"%PDF-1.4\n{content}\n%%EOF", "utf-8"))

def send_email_with_attachment():
    subject = "AUTO_TEST_MESSAGE"
    body = generate_random_text(50)
    pdf_name = "test_file.pdf"
    create_pdf(pdf_name)

    msg = EmailMessage()
    msg["From"] = EMAIL_ADDRESS
    msg["To"] = EMAIL_ADDRESS
    msg["Subject"] = subject
    msg.set_content(body)

    with open(pdf_name, "rb") as f:
        msg.add_attachment(
            f.read(),
            maintype="application",
            subtype="pdf",
            filename=pdf_name
        )

    with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.send_message(msg)

def auto_reply_to_subject(search_subject):
    imap = imaplib.IMAP4_SSL(IMAP_SERVER)
    imap.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    imap.select("INBOX")

    status, messages = imap.search(None, f'(SUBJECT "{search_subject}")')

    for num in messages[0].split():
        status, data = imap.fetch(num, "(RFC822)")
        msg = EmailMessage()
        raw_email = data[0][1]
        msg = email.message_from_bytes(raw_email)

        sender = parseaddr(msg.get("From"))[1]

        reply = EmailMessage()
        reply["From"] = EMAIL_ADDRESS
        reply["To"] = sender
        reply["Subject"] = "Автоматический ответ"
        reply.set_content("Ваше сообщение получено и обработано автоматически.")

        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.send_message(reply)
    imap.close()
    imap.logout()

send_email_with_attachment()
auto_reply_to_subject("AUTO_TEST_MESSAGE")
